# VacationPy

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.83,83,100,8.46,PN,1699827225
1,1,jilotepec de molina enriquez,19.9528,-99.5347,20.65,45,43,2.82,MX,1699827400
2,2,yichang,30.7144,111.2847,7.69,87,89,0.40,CN,1699827401
3,3,kruisfontein,-34.0033,24.7314,11.82,85,2,3.76,ZA,1699827401
4,4,invercargill,-46.4000,168.3500,11.68,95,100,2.68,NZ,1699827401


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
   

)
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Define your criteria for an ideal weather condition
max_temp_criteria = 25
min_temp_criteria = 20
max_humidity = 30 
max_cloudiness = 10
max_wind_speed = 10 

# Filter the DataFrame based on the criteria
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] < max_temp_criteria) &
    (city_data_df['Max Temp'] > min_temp_criteria) &
    (city_data_df['Humidity'] < max_humidity) &
    (city_data_df['Cloudiness'] < max_cloudiness) &
    (city_data_df['Wind Speed'] < max_wind_speed)
]

# Drop rows with null values (if any)
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data from the filtered DataFrame
ideal_cities_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
229,229,arroyito,-31.4202,-63.0500,22.28,29,0,7.13,AR,1699827483
364,364,bardai,11.6500,7.2333,23.20,24,2,3.01,NG,1699827262
429,429,zouerate,22.7187,-12.4521,24.03,11,0,2.98,MR,1699827565
548,548,alaghsas,17.0187,8.0168,24.81,14,0,5.73,NE,1699827637
576,576,agadez,19.7500,10.2500,22.85,17,0,4.35,NE,1699827650


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Create a new DataFrame 'hotel_df' as a copy of 'ideal_cities_df' with selected columns
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column 'Hotel Name'
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
229,arroyito,AR,-31.4202,-63.0500,29,
364,bardai,NG,11.6500,7.2333,24,
429,zouerate,MR,22.7187,-12.4521,11,
548,alaghsas,NE,17.0187,8.0168,14,
576,agadez,NE,19.7500,10.2500,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    "type": "lodging", 
    "radius": radius,  
    "limit": 1, 
    "apiKey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set the base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
arroyito - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
alaghsas - nearest hotel: No hotel found
agadez - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
229,arroyito,AR,-31.4202,-63.0500,29,No hotel found
364,bardai,NG,11.6500,7.2333,24,No hotel found
429,zouerate,MR,22.7187,-12.4521,11,No hotel found
548,alaghsas,NE,17.0187,8.0168,14,No hotel found
576,agadez,NE,19.7500,10.2500,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)